In [ ]:
import psycopg2
from datetime import date
import tkinter as tk

# Connect to the PostgreSQL database
def connect_to_database():
    connection = psycopg2.connect(
        host="localhost",
        port="5432",
        database="postgres",
        user="postgres",
        password="1234"
    )
    return connection

# Create the products table in the database if it doesn't exist
def create_table(connection):
    cursor = connection.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS products (
            name VARCHAR(255) PRIMARY KEY,
            purchase_year INTEGER,
            warranty_duration INTEGER,
            location VARCHAR(255)
        )
    """)
    connection.commit()

# Add a product to the database
def add_product():
    name = name_entry.get()
    purchase_year = int(purchase_year_entry.get())
    warranty_duration = int(warranty_duration_entry.get())
    location = location_entry.get()

    cursor = connection.cursor()
    cursor.execute("""
        INSERT INTO products (name, purchase_year, warranty_duration, location)
        VALUES (%s, %s, %s, %s)
    """, (name, purchase_year, warranty_duration, location))
    connection.commit()
    result_label.config(text="Product added successfully.")

# Check the warranty status of a product
def check_warranty():
    name = name_entry.get()

    cursor = connection.cursor()
    cursor.execute("""
        SELECT purchase_year, warranty_duration
        FROM products
        WHERE name = %s
    """, (name,))
    product = cursor.fetchone()

    if product is not None:
        purchase_year, warranty_duration = product
        warranty_expiry_year = purchase_year + warranty_duration
        current_year = date.today().year

        warranty_status = "under warranty" if current_year <= warranty_expiry_year else "expired"
        result_label.config(text=f"The {name} product is {warranty_status}.")
    else:
        result_label.config(text="Product not found.")

# Get the location of a product
def get_location():
    name = name_entry.get()

    cursor = connection.cursor()
    cursor.execute("""
        SELECT location
        FROM products
        WHERE name = %s
    """, (name,))
    location = cursor.fetchone()

    if location is not None:
        result_label.config(text=f"The {name} is located at: {location[0]}")
    else:
        result_label.config(text="Product not found.")

# Connect to the database
connection = connect_to_database()
create_table(connection)

# Create Tkinter GUI
window = tk.Tk()
window.title("Product Management")

# Create input fields and labels
name_label = tk.Label(window, text="Product Name:")
name_label.pack()
name_entry = tk.Entry(window)
name_entry.pack()

purchase_year_label = tk.Label(window, text="Year of Purchase:")
purchase_year_label.pack()
purchase_year_entry = tk.Entry(window)
purchase_year_entry.pack()

warranty_duration_label = tk.Label(window, text="Warranty Duration (years):")
warranty_duration_label.pack()
warranty_duration_entry = tk.Entry(window)
warranty_duration_entry.pack()

location_label = tk.Label(window, text="Product Location:")
location_label.pack()
location_entry = tk.Entry(window)
location_entry.pack()

# Create result label
result_label = tk.Label(window, text="")
result_label.pack()

# Create buttons
add_button = tk.Button(window, text="Add Product", command=add_product)
add_button.pack()

check_button = tk.Button(window, text="Check Warranty", command=check_warranty)
check_button.pack()

location_button = tk.Button(window, text="Get Location", command=get_location)
location_button.pack()

window.mainloop()

# Close the database connection
connection.close()
